In [ ]:
!pip install datasets transformers rouge-score nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 86.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 103.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.6 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=3fcaac6d633de3f5b2a0d9cd04b2fea46b1700e72f0c27b28891338e7a88d885
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a7

In [ ]:
!pip install --upgrade debugpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 28.0 MB/s eta 0:00:00
  Attempting uninstall: debugpy
    Found existing installation: debugpy 1.6.6
    Uninstalling debugpy-1.6.6:
      Successfully uninstalled debugpy-1.6.6


In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# !apt install git-lfs

In [ ]:
import transformers

print(transformers.__version__)

4.34.0


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
pip install accelerate -U

In [ ]:
pip install -r requirements.txt -r requirements-heavy.txt


NotImplementedError: ignored

In [ ]:
!pip install --no-deps 'gem-metrics @ git+https://github.com/GEM-benchmark/GEM-metrics.git'


  Cloning https://github.com/GEM-benchmark/GEM-metrics.git to /tmp/pip-install-xtpgrm8e/gem-metrics_ed7571b7e5034d7da53810780c7464a1
  Running command git clone --filter=blob:none --quiet https://github.com/GEM-benchmark/GEM-metrics.git /tmp/pip-install-xtpgrm8e/gem-metrics_ed7571b7e5034d7da53810780c7464a1
  Resolved https://github.com/GEM-benchmark/GEM-metrics.git to commit 8162210e73728c68f886cad6c09160694e3be9d9
  Preparing metadata (setup.py) ... done
  Created wheel for gem-metrics: filename=gem_metrics-0.1.dev0-py3-none-any.whl size=63328 sha256=9ace25a7384943a36f21ea2b3b1989bc6ca0fe7f0585a95b786ec4d59dbbd13f
  Stored in directory: /tmp/pip-ephem-wheel-cache-eet3iqee/wheels/6c/53/96/9e4f08e08c4f0d4905235b50376952771cd43a445862ee9804
Successfully built gem-metrics


# Fine-tuning a model on a summarization task

In [ ]:
model_checkpoint = "t5-base"

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [ ]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("eli5")
metric = load_metric("rouge")

Downloading:   0%|          | 0.00/3.50k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/576M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/21.1M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/53.0M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/286M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/330M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/18.7M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/36.2M [00:00<?, ?B/s]

<ipython-input-11-9c1c30cfab92>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [ ]:
raw_datasets

DatasetDict({
    train_eli5: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'answers', 'title_urls', 'selftext_urls', 'answers_urls'],
        num_rows: 272634
    })
    validation_eli5: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'answers', 'title_urls', 'selftext_urls', 'answers_urls'],
        num_rows: 9812
    })
    test_eli5: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'answers', 'title_urls', 'selftext_urls', 'answers_urls'],
        num_rows: 24512
    })
    train_asks: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'answers', 'title_urls', 'selftext_urls', 'answers_urls'],
        num_rows: 131778
    })
    validation_asks: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'answers', 'title_urls', 'selftext_urls', 'answers_urls'],
        num_rows: 2281
    })
    test_asks: Dataset({
       

To access an actual element, you need to select a split first, then give an index:

In [ ]:
raw_datasets["train_eli5"][1]

{'q_id': '2lojul',
 'title': "Why are different tiers (regular < mid < premium) of gas' prices almost always 10 cents different?",
 'selftext': "I've noticed that the difference in price between regular gas and midrange, and between midrange and premium, is almost always 10 cents. This seems to hold true no matter what the price for regular gas. This doesn't seem to make sense, as the difference between $2 and $2.10 and the difference between $4 and $4.10 /gal are proportionally very different. Is this just an arbitrary convention that undermines arguments of a rational basis for gasoline prices? ",
 'document': '',
 'subreddit': 'explainlikeimfive',
 'answers': {'a_id': ['clwqgxl', 'clwqpjq', 'clwuh3s'],
  'text': ['As someone who uses quality Premium, I wish this was true.',
   "The difference is in how it burns though is what's critical for you as the end consumer. I drive a forced induction car, so air coming into my engine is compressed before it enters the cylinder where it's fur

## Preprocessing the data

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:158: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [ ]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc + " " + text for doc,text in zip(examples["title"], examples["selftext"])]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer([answer["text"][0] for answer in examples["answers"]], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocess_function(raw_datasets['train_eli5'][:2])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3864: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': [[21603, 10, 16, 3370, 125, 7, 8, 500, 13, 3, 26281, 8, 166, 192, 4805, 28, 3, 9, 10505, 3, 18, 95, 8, 2214, 3, 18, 59, 1646, 10505, 4805, 3, 23, 129, 273, 1], [21603, 10, 1615, 33, 315, 3, 3276, 7, 41, 60, 122, 4885, 3, 2, 2076, 3, 2, 3331, 61, 13, 1807, 31, 1596, 966, 373, 335, 3151, 7, 315, 58, 27, 31, 162, 4944, 24, 8, 1750, 16, 594, 344, 1646, 1807, 11, 2076, 5517, 6, 11, 344, 2076, 5517, 11, 3331, 6, 19, 966, 373, 335, 3151, 7, 5, 100, 1330, 12, 1520, 1176, 150, 1052, 125, 8, 594, 21, 1646, 1807, 5, 100, 744, 31, 17, 1727, 12, 143, 1254, 6, 38, 8, 1750, 344, 3771, 11, 3771, 5, 1714, 11, 8, 1750, 344, 7514, 11, 7514, 5, 1714, 3, 87, 6191, 33, 7385, 1427, 182, 315, 5, 27, 7, 48, 131, 46, 3, 26968, 8346, 24, 23248, 7, 12874, 13, 3, 9, 12226, 1873, 21, 24456, 1596, 58, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/272634 [00:00<?, ? examples/s]

Map:   0%|          | 0/9812 [00:00<?, ? examples/s]

Map:   0%|          | 0/24512 [00:00<?, ? examples/s]

Map:   0%|          | 0/131778 [00:00<?, ? examples/s]

Map:   0%|          | 0/2281 [00:00<?, ? examples/s]

Map:   0%|          | 0/4462 [00:00<?, ? examples/s]

Map:   0%|          | 0/98525 [00:00<?, ? examples/s]

Map:   0%|          | 0/4901 [00:00<?, ? examples/s]

Map:   0%|          | 0/9764 [00:00<?, ? examples/s]

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the `AutoModelForSeq2SeqLM` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
batch_size = 4
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    num_train_epochs=2,
    predict_with_generate=True,
    output_dir = "/content/drive/MyDrive/Colab/eli5",
    save_strategy = "epoch"


)

ImportError: ignored

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

The last thing to define for our `Seq2SeqTrainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

Then we just need to pass all of this along with our datasets to the `Seq2SeqTrainer`:

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train_eli5"],
    eval_dataset=tokenized_datasets["validation_eli5"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

NameError: ignored

We can now finetune our model by just calling the `train` method:

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: selftext, title, subreddit, title_urls, answers, selftext_urls, answers_urls, q_id, document. If selftext, title, subreddit, title_urls, answers, selftext_urls, answers_urls, q_id, document are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 272634
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 136318
  Number of t

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,3.249300,3.097203,14.561000,2.367000,11.314900,13.149500,18.981800
2,3.220700,3.077972,14.606600,2.391800,11.343200,13.181300,18.982700


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: selftext, title, subreddit, title_urls, answers, selftext_urls, answers_urls, q_id, document. If selftext, title, subreddit, title_urls, answers, selftext_urls, answers_urls, q_id, document are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9812
  Batch size = 4
Saving model checkpoint to /content/drive/MyDrive/Colab/eli5/checkpoint-68159
Configuration saved in /content/drive/MyDrive/Colab/eli5/checkpoint-68159/config.json
Model weights saved in /content/drive/MyDrive/Colab/eli5/checkpoint-68159/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Colab/eli5/checkpoint-68159/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Colab/eli5/checkpoint-68159/special_tokens_map.json
The following columns in the evaluatio

TrainOutput(global_step=136318, training_loss=3.2653891931780437, metrics={'train_runtime': 42394.8748, 'train_samples_per_second': 12.862, 'train_steps_per_second': 3.215, 'total_flos': 6.310382279233536e+16, 'train_loss': 3.2653891931780437, 'epoch': 2.0})

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

OSError: ignored

In [ ]:
from transformers import pipeline

model_name = 'Muzzi/t5-base-finetuned-eli5'
pipe = pipeline('text2text-generation', model=model_name, tokenizer=model_name)

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Model device: ", next(model.parameters()).device)
model.to(device)


test_data = tokenized_datasets['test_eli5']


Model device:  cpu


In [ ]:
pip install spacy

In [ ]:
generated_answers = []
for example in test_data:
    input_ids = torch.tensor(example['input_ids']).to(device).unsqueeze(0)
    output = model.generate(input_ids)
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
    generated_answers.append(decoded_output)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
with open('generated_answers.txt', 'w',encoding='utf-8') as file:
    for answer in generated_answers:
        file.write(answer + '\n')

In [ ]:
from google.colab import files

files.download('generated_answers.txt')  # or 'generated_answers.json' for the JSON file

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>